In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline
plt.gray();
from matplotlib.pyplot import imshow

import skimage
from skimage.io import imread, imsave
from skimage.transform import resize

from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display

In [ ]:
import requests
def download(url, filename):
    with open(filename, 'wb') as saveFile:
        saveFile.write(requests.get(url).content)
    
# TheDefiniteArticle - Own work CC BY-SA 4.0
# https://commons.wikimedia.org/wiki/File:Woy_Woy_Channel_-_Vignetted.jpg
download(
    'https://raw.githubusercontent.com/tttamaki/lecture_code/main/ImageProcessing/images/lake.jpg',
    # 'https://upload.wikimedia.org/wikipedia/commons/0/03/Woy_Woy_Channel_-_Vignetted.jpg',
    'lake.jpg')

# Pixabay License
# https://pixabay.com/photos/girl-portrait-looking-young-female-919048/
download(
    'https://raw.githubusercontent.com/tttamaki/lecture_code/main/ImageProcessing/images/girl.jpg',
    # 'https://cdn.pixabay.com/photo/2015/09/02/13/24/girl-919048_1280.jpg',
    'girl.jpg')


# Diliff - Own work CC BY-SA 2.5
# http://en.wikipedia.org/wiki/File:Colosseum_in_Rome,_Italy_-_April_2007.jpg
download(
    'https://raw.githubusercontent.com/tttamaki/lecture_code/main/ImageProcessing/images/Colosseum.jpg',
    # 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Colosseum_in_Rome%2C_Italy_-_April_2007.jpg/640px-Colosseum_in_Rome%2C_Italy_-_April_2007.jpg',
    'Colosseum.jpg')
download(
    'https://raw.githubusercontent.com/tttamaki/lecture_code/main/ImageProcessing/images/Colosseum_original.jpg',
    # 'https://upload.wikimedia.org/wikipedia/commons/5/53/Colosseum_in_Rome%2C_Italy_-_April_2007.jpg',
    'Colosseum_original.jpg')

# CC0 1.0
# https://openclipart.org/detail/247372/woman-walking
download(
    'https://raw.githubusercontent.com/tttamaki/lecture_code/main/ImageProcessing/images/sil.png',
    # 'https://openclipart.org/image/400px/svg_to_png/247372/WomanWalking.png',
    'sil.png')

# NASA Public domain
# File:Moon right-view (Clementine dataset).png
download(
    'https://raw.githubusercontent.com/tttamaki/lecture_code/main/ImageProcessing/images/moon.png',
    # 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Moon_right-view_%28Clementine_dataset%29.png/800px-Moon_right-view_%28Clementine_dataset%29.png',
    'moon.png')

# 写真のExif

In [ ]:
def show_exif(filename):
    im = Image.open(filename)
    exif = im._getexif()
    if exif is None:
        print('no EXIF data')
    else:
        for i, value in exif.items():
            print(i, TAGS.get(i), value)

In [ ]:
imshow(imread('lake.jpg'));

In [ ]:
show_exif('lake.jpg')

In [ ]:
imshow(imread('girl.jpg'));

In [ ]:
show_exif('girl.jpg')

In [ ]:
imshow(imread('Colosseum.jpg'));

In [ ]:
show_exif('Colosseum.jpg')

In [ ]:
imshow(imread('Colosseum_original.jpg'));

In [ ]:
show_exif('Colosseum_original.jpg')

# 周辺光量低下

In [ ]:
im = imread('girl.jpg')
imshow(im)
plt.axis('off')
plt.show()

In [ ]:
h, w, _ = im.shape
im2 = np.zeros_like(im)


@interact(coeff=(0, 5, 1))
def g(coeff=3):
    
    for y in range(h):
        cy = h//2 - y
        for x in range(w):
            cx = w//2 - x
            r2 = cx**2 + cy**2
            r4 = r2**2
            im2[y, x] = im[y, x] * (1 - coeff * 1e-12 * r4)

    imshow(im2)
    plt.axis('off')
    plt.title('coeff={}'.format(coeff))
    plt.show()


# 歪曲収差

In [ ]:
h, w, _ = im.shape


@interact(coeff=(-9, -3, 1),
          ud=RadioButtons(options=[1, -1]))
def g(coeff=-10, ud=1):
    
    im2 = np.zeros_like(im)

    for y in range(h):
        yu = h//2 - y
        for x in range(w):
            xu = w//2 - x
            r2 = xu**2 + yu**2
            c = 1 + ud * r2 * (10 ** coeff)
            yd = int(h//2 - yu * c)
            xd = int(w//2 - xu * c)
            if 0 <= yd < h and 0 <= xd < w:
                im2[y, x] = im[yd, xd]

    imshow(im2)
    plt.axis('off')
    plt.title('coeff=1e{}'.format(coeff))
    plt.show()



# 透視投影パラメータの算出

## 問題：月と人物

月と人物が重なる映像を撮影したい．

In [ ]:
sil = imread('sil.png')

In [ ]:
moon = imread('moon.png')

In [ ]:
hs, ws = sil.shape[:2]
hm, wm = moon.shape[:2]
scale = hm / hs

In [ ]:
sil = resize(sil, ( (int(hs * scale + 0.5), int(ws * scale + 0.5)))).astype(np.uint8)
hs, ws = sil.shape[:2]

In [ ]:
moon[:, wm//2 - ws//2:wm//2 + ws//2, 3] *= (1 - sil[:, :, 3])

In [ ]:
imshow(moon)
plt.axis('off')
plt.show()

このような映像を撮影するための条件を考える．

以下を仮定する．
- 人物の身長は$H=170$ [cm]
- 人物の画面中の高さ$h$は，画面の高さ$h$と同じ
- 月の画面中の高さ$h$も，画面の高さ$h$と同じ

月の半径と月までの距離は各自で調べよ．

カメラから人物までの距離を$L_1$ [m]とする．
撮影に使用するカメラのレンズの焦点距離を$f$ [mm]，センササイズは35mmフィルム換算とする（つまりセンサ高は$h=24$ [mm]）．


- $L_1$ [m]と$f$ [mm]を求めよ．
- 求めた$f$に近いレンズ焦点距離を持つ一眼レフカメラ用のレンズを<a href="https://www.amazon.co.jp">Amazon.co.jp</a>から探せ．これが撮影に利用する機材候補である．

## 問題：マラソン中継カメラ

Katie Chan - Own work CC BY-SA 4.0
https://commons.wikimedia.org/wiki/File:2017_London_Marathon_-_Alphonce_Simbu.jpg

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/2017_London_Marathon_-_Alphonce_Simbu.jpg/320px-2017_London_Marathon_-_Alphonce_Simbu.jpg">

マラソン中継において，上の写真のようにトップランナー前方から後方に撮影したとき，第2集団の選手は画面の中ではトップランナーに比べて小さく写ってしまう．
そうならないためにはどのような撮影をすればよいかを考える．

以下を仮定する．
- 選手の身長はすべて$H$=170 [cm]
- トップランナーと第2集団との差は10秒．選手は1kmを3分で走る．
- すべての選手は画面の中央に写る（つまり選手の縦中央は画面の縦中央）．
- トップランナーの画面中の高さ$h_1$は，画面の高さ$h$の1/3を占める
- 第2集団の選手の画面中の高さ$h_2$は$h_2 = \alpha h_1$とする．ここで$0 < \alpha < 1$であり，番組プロデューサは$\alpha=0.8$を希望している

中継カメラからトップランナーまでの距離を$L_1$ [m]，トップランナーと第2集団との間の距離を$L_2$ [m]とする．
撮影に使用するカメラのレンズの焦点距離を$f$ [mm]，センササイズは35mmフィルム換算とする（つまりセンサ高は$h=24$ [mm]）．


- $L_1$ [m]と$f$ [mm]を求めよ．
- 求めた$f$に近いレンズ焦点距離を持つ一眼レフカメラ用のレンズを<a href="https://www.amazon.co.jp">Amazon.co.jp</a>から探せ．これが撮影に利用する機材候補である（本当は放送映像用レンズだがここでは一眼レフカメラにする）．カメラ本体はすでにあるものを利用し，レンズだけを新規に購入するとする．このレンズを購入するための予算をいくつか設定し，予算を超えてしまう場合には，どのパラメータを諦めれば予算内に収まるのかを提案せよ．